In [1]:
cd "/content/drive/MyDrive/CS544-NLP/project"

[Errno 2] No such file or directory: '/content/drive/MyDrive/CS544-NLP/project'
/kaggle/working


In [2]:
!pip install -q -U datasets

In [3]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!pip install -q git+https://github.com/huggingface/transformers.git@main accelerate

### Loading the model and tokenizer

In [5]:
import datasets

In [6]:
from transformers import AutoTokenizer
import transformers
import torch
from tqdm import notebook
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
model_id = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id)

### Preparing the Pipeline

In [8]:
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

In [9]:
def tokenize(data):
    result = tokenizer(
        data['prompt'],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [10]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
pipeline.tokenizer.pad_token_id = tokenizer.eos_token_id

### Generating Code

In [12]:
def llama_v2_prompt(
    messages: list[dict]
):
    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    BOS, EOS = "<s>", "</s>"
    DEFAULT_SYSTEM_PROMPT = f"""Provide answers in Python. Wrap around code in [PYTHON] and [/PYTHON]. Don't write any comments in generated code."""

    if messages[0]["role"] != "system":
        messages = [
            {
                "role": "system",
                "content": DEFAULT_SYSTEM_PROMPT,
            }
        ] + messages


    messages = [
        {
            "role": messages[1]["role"],
            "content": B_SYS + messages[0]["content"] + E_SYS + messages[1]["content"],
        }
    ] + messages[2:]

    messages_list = [
        f"{BOS}{B_INST}{(prompt['content']).strip()}{E_INST}{(answer['content']).strip()}{EOS}"
        for prompt, answer in zip(messages[::2], messages[1::2])
    ]
    messages_list.append(f"{BOS}{B_INST}{(messages[-1]['content']).strip()}{E_INST}\n")

    return "".join(messages_list)

In [13]:
def generate_code(dataset):
    responses = []
    batch_count = 0
    batch_size = 2
    print(f"Total batches = {len(dataset)//batch_size}")
    for out in pipeline(
      KeyDataset(dataset, "prompt"),
      do_sample=True,
      temperature=0.1,
      top_p=0.9,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=512,
      batch_size = batch_size
      ):
        batch_count += 1
        for o in out:
              responses.append(o['generated_text'])
        print('Processed batch[%d]'%batch_count, end='\r')
    return responses

In [14]:
def end_overlap(a, b):
    for i in range(0, len(a)):
        if b.startswith(a[i:]):
            return i
    return len(a)

def remove_prefix(s, prefix):
    prefix_end_idx = s.index(prefix) + len(prefix)
    return s[prefix_end_idx:]

def extract_code(prompt, response, code_context):
    B_PYTHON, E_PYTHON = '[PYTHON]', '[/PYTHON]'
    trimmed_response = remove_prefix(response, prompt)
    if trimmed_response.startswith(B_PYTHON):
        trimmed_response = remove_prefix(trimmed_response, B_PYTHON)
    try:
        suffix_idx = trimmed_response.index(E_PYTHON)
        trimmed_response = trimmed_response[:suffix_idx]
    except ValueError as ve:
          pass
    trimmed_response = trimmed_response.strip()
    code_context = code_context.strip()
    overlap_idx = end_overlap(code_context, trimmed_response)
    cleaned_response = code_context[:overlap_idx] + trimmed_response
    return cleaned_response

def save_file(path, code):
    with open(path, 'w') as file:
        file.write(code)

def run_code_generation(dataset, save_path, n_passes=5):
    responses = generate_code(dataset)
    return responses
  # for index, response in responses:
  #   question = row['question']
  #   code_context = row['code_context']
  #   for p in range(n_passes):
  #     prompt = f"# {question}\n{code_context}\n"
  #     instruction = {
  #       "role": "user",
  #       "content": prompt,
  #     }
  #     prompt, response = generate_code([instruction])
  #     code = extract_code(prompt, response, code_context)
  #     save_file(f"{save_path}/question-{index}-pass-{p}.py", code)

In [15]:

response = """<s>[INST]<<SYS>>
Provide answers in Python. Wrap around code in [PYTHON] and [/PYTHON]. Don't write any comments in generated code.
<</SYS>>

# Maximum Prefix Sum possible by merging two given arrays.
def maxPresum(a, b):[/INST]
[PYTHON]
def maxPresum(a, b):
    max_sum = 0
    for i in range(len(a)):
        for j in range(len(b)):
            max_sum = max(max_sum, a[i] + b[j])
    return max_sum
[/PYTHON]
"""

prompt = """<s>[INST]<<SYS>>
Provide answers in Python. Wrap around code in [PYTHON] and [/PYTHON]. Don't write any comments in generated code.
<</SYS>>

# Maximum Prefix Sum possible by merging two given arrays.
def maxPresum(a, b):[/INST]
"""

code_context = """def maxPresum(a, b):
"""

In [16]:
# extract_code(prompt, response, code_context)

In [17]:
CODE_GENERATION_PATH = "./generations/python"

In [18]:
train_df = pd.read_csv('/kaggle/input/xlcost-text-to-code/train_processed.csv', index_col=0)
train_df.head()

,text,code,question,functions,code_context
id,,,,,
0,Maximum Prefix Sum possible by merging two giv...,"def maxPresum(a, b):\n X = max(a[0], 0)\n ...",Maximum Prefix Sum possible by merging two giv...,"['def maxPresum(a, b):']","def maxPresum(a, b):"
1,Check if a number can be represented as sum of...,import math\n\n\ndef sumOfTwoCubes(n):\n lo...,Check if a number can be represented as sum of...,['def sumOfTwoCubes(n):'],import math\n\n\ndef sumOfTwoCubes(n):
2,Generate an N | Python3 program for the above ...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,Generate an N.,"['def sieveOfPrimes():', 'def getArray(arr, N):']",sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...
3,Nth natural number after removing all numbers ...,def findNthNumber(N):\n result = 0\n p =...,Nth natural number after removing all numbers ...,['def findNthNumber(N):'],def findNthNumber(N):
4,Check if an integer is rotation of another giv...,"import math\n\n\ndef check(A, B):\n if (A =...",Check if an integer is rotation of another giv...,"['def check(A, B):']","import math\n\n\ndef check(A, B):"


In [19]:
def build_prompt(question, code_context):
    prompt = f"# {question}\n{code_context}\n"
    instruction = {
      "role": "user",
      "content": prompt,
    }
    llama_prompt = llama_v2_prompt([instruction])
    return llama_prompt

In [20]:
def build_question_context(text, context):
    q_strs = text.split('|')
    q_strs = q_strs[1].split(';')[1:]
    # comments = []
    # for i in range(1, len(q_strs)):
    #     comments.append(f'\\ {i}. {q_strs[i]}')

    # return ' '.join(comments) + '\n' + context
    return context

In [21]:
train_df_300 = train_df[0:300].copy()
train_df_300['context'] = train_df_300.apply(lambda row: build_question_context(row.text, row.code_context), axis=1)
train_df_300['llama_prompt'] = train_df_300.apply(lambda row: build_prompt(row.question, row.code_context), axis=1)
train_df_300.head()

,text,code,question,functions,code_context,context,llama_prompt
id,,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,"def maxPresum(a, b):\n X = max(a[0], 0)\n ...",Maximum Prefix Sum possible by merging two giv...,"['def maxPresum(a, b):']","def maxPresum(a, b):","def maxPresum(a, b):",<s>[INST]<<SYS>>\nProvide answers in Python. W...
1,Check if a number can be represented as sum of...,import math\n\n\ndef sumOfTwoCubes(n):\n lo...,Check if a number can be represented as sum of...,['def sumOfTwoCubes(n):'],import math\n\n\ndef sumOfTwoCubes(n):,import math\n\n\ndef sumOfTwoCubes(n):,<s>[INST]<<SYS>>\nProvide answers in Python. W...
2,Generate an N | Python3 program for the above ...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,Generate an N.,"['def sieveOfPrimes():', 'def getArray(arr, N):']",sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
3,Nth natural number after removing all numbers ...,def findNthNumber(N):\n result = 0\n p =...,Nth natural number after removing all numbers ...,['def findNthNumber(N):'],def findNthNumber(N):,def findNthNumber(N):,<s>[INST]<<SYS>>\nProvide answers in Python. W...
4,Check if an integer is rotation of another giv...,"import math\n\n\ndef check(A, B):\n if (A =...",Check if an integer is rotation of another giv...,"['def check(A, B):']","import math\n\n\ndef check(A, B):","import math\n\n\ndef check(A, B):",<s>[INST]<<SYS>>\nProvide answers in Python. W...


In [22]:
prompts = train_df_300['llama_prompt'].values.tolist()
prompts_dic = {'prompt': prompts}
prompts_dataset = Dataset.from_dict(prompts_dic)

In [23]:
prompts_dataset[0]

{'prompt': "<s>[INST]<<SYS>>\nProvide answers in Python. Wrap around code in [PYTHON] and [/PYTHON]. Don't write any comments in generated code.\n<</SYS>>\n\n# Maximum Prefix Sum possible by merging two given arrays.\ndef maxPresum(a, b):[/INST]\n"}

In [24]:
responses = run_code_generation(prompts_dataset, CODE_GENERATION_PATH, n_passes=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Total batches = 150


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [25]:
len(responses)

300

In [26]:
train_df_300['response'] = responses

train_df_300.head()

,text,code,question,functions,code_context,context,llama_prompt,response
id,,,,,,,,
0,Maximum Prefix Sum possible by merging two giv...,"def maxPresum(a, b):\n X = max(a[0], 0)\n ...",Maximum Prefix Sum possible by merging two giv...,"['def maxPresum(a, b):']","def maxPresum(a, b):","def maxPresum(a, b):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
1,Check if a number can be represented as sum of...,import math\n\n\ndef sumOfTwoCubes(n):\n lo...,Check if a number can be represented as sum of...,['def sumOfTwoCubes(n):'],import math\n\n\ndef sumOfTwoCubes(n):,import math\n\n\ndef sumOfTwoCubes(n):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
2,Generate an N | Python3 program for the above ...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,Generate an N.,"['def sieveOfPrimes():', 'def getArray(arr, N):']",sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,sieve = [1] * (1000000 + 1)\n\n\ndef sieveOfPr...,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
3,Nth natural number after removing all numbers ...,def findNthNumber(N):\n result = 0\n p =...,Nth natural number after removing all numbers ...,['def findNthNumber(N):'],def findNthNumber(N):,def findNthNumber(N):,<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...
4,Check if an integer is rotation of another giv...,"import math\n\n\ndef check(A, B):\n if (A =...",Check if an integer is rotation of another giv...,"['def check(A, B):']","import math\n\n\ndef check(A, B):","import math\n\n\ndef check(A, B):",<s>[INST]<<SYS>>\nProvide answers in Python. W...,<s>[INST]<<SYS>>\nProvide answers in Python. W...


In [27]:
train_df_300.to_csv('/kaggle/working/train_response.csv')

### Testing on Several Queries

### Code Infilling

For Future...

In [28]:
# from transformers import pipeline
# import torch

# generator = pipeline("text-generation",model="codellama/CodeLlama-7b-hf",torch_dtype=torch.float16, device_map="auto")
# # generator('def remove_non_ascii(s: str) -> str:\n    """ <FILL_ME>\n    return result', max_new_tokens = 128, return_type = 1)

In [29]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import transformers
# import torch

# model_id = "codellama/CodeLlama-7b-hf"
# tokenizer2 = AutoTokenizer.from_pretrained(model_id)
# model2 = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16
# ).to("cuda")




In [30]:
# prompt = '''def remove_non_ascii(s: str) -> str:
#     """ <FILL_ME>
#     return result
# '''

# input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to("cuda")
# output = model.generate(
#     input_ids,
#     max_new_tokens=200,
# )
# output = output[0].to("cpu")

# filling = tokenizer.decode(output[input_ids.shape[1]:], skip_special_tokens=True)


In [31]:
# print(prompt.replace("<FILL_ME>", filling))